In [1]:
import torch
from torch2trt import torch2trt
from imitation.algorithms import bc
from imitation.util import util



In [3]:
import os
import json
import numpy as np
from PIL import Image

def load_expert_data(data_path):
    # expert_dataの初期化
    expert_data = {'images': [], 'actions': []}
    # print(expert_data)
    # count = 0

    # 画像ファイルに対応するJSONファイルを取得
    json_file_list = [json_file for json_file in os.listdir(data_path) if json_file.startswith('record_') and json_file.endswith('.json')]
    # print(len(json_file_list))

    for json_file in json_file_list:
        # count += 1
        # print(count)
        # print(json_file)

        # レコードのファイルパスを構築
        json_path = os.path.join(data_path, json_file)
        # print(json_path)

        # レコードの読み込み
        try:
            with open(json_path, 'r') as json_file:
                record_data = json.load(json_file)
        except FileNotFoundError:
            print(f"エラー：{json_path} でJSONファイルが見つかりませんでした。")
            continue
        except json.JSONDecodeError:
            print(f"エラー：{json_path} のJSONファイルのデコードに失敗しました。")
            continue

        # 画像データの読み込み
        image_file = record_data.get('cam/image_array', '')  # 画像ファイル名をJSONから取得
        # print(image_file)
        image_path = os.path.join(data_path, image_file)
        # print(image_path)
        try:
            image_data = np.array(Image.open(image_path))
        except FileNotFoundError:
            print(f"エラー：{image_path} で画像ファイルが見つかりませんでした。")
            continue

        # expert_dataに追加
        expert_data['images'].append(image_data)
        expert_data['actions'].append([record_data.get('user/angle', 0), record_data.get('user/throttle', 0)])

    return expert_data


In [4]:
# expert_data_path = '../../Data/autorace/O/tub_9_24-01-09'
expert_data_path = '../../autorace/data/tub_9_24-01-09'
expert_data = load_expert_data(expert_data_path)
print(expert_data['images'][0].shape)
reshaped_expert_data = expert_data
# すべての画像を変換
for i in range(len(reshaped_expert_data['images'])):
    # 画像の軸の順序を変更
    reshaped_image = np.transpose(reshaped_expert_data['images'][i], (2, 0, 1))
    
    # expert_dataに変更を反映
    reshaped_expert_data['images'][i] = reshaped_image

# 形状を確認
print(reshaped_expert_data['images'][0].shape)

for i in range(len(reshaped_expert_data['images'])):
    if reshaped_expert_data['images'][i].shape != (3, 224, 224):
        print(i)



(224, 224, 3)
(3, 224, 224)


In [6]:
# 学習済みのBCモデルを読み込む
bc_trainer = bc.reconstruct_policy('./model/bc_trainer_policy')

# 保存したpthファイルをtrtファイルに変換
dummy_input = torch.randn(1, 3, 224, 224).cuda()
# trt_model = torch2trt(bc_trainer, [dummy_input], fp16_mode=True)
trt_model = torch2trt(bc_trainer, [dummy_input], fp16_mode=True, max_workspace_size=1<<25)



In [7]:
torch.backends.cuda.CUDA_LAUNCH_BLOCKING = 1


In [8]:
observation_tensor = torch.tensor(reshaped_expert_data['images'][12], dtype=torch.float32)
observation_tensor = observation_tensor.unsqueeze(0)
action = trt_model(observation_tensor)
torch.backends.cuda.CUDA_LAUNCH_BLOCKING=1
print(action)

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


: 

In [1]:
import torch

# 利用可能なCUDAメモリを確認
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_cached())

# CUDAメモリの解放
torch.cuda.empty_cache()

# 解放後の利用可能なCUDAメモリを再確認
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_cached())


0
0
0
0


/home/gaia-22/miniconda3/envs/env38/lib/python3.8/site-packages/torch/cuda/memory.py:444: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


2.1.2+cu121
12.1
